In [ ]:
from glob import glob

In [ ]:
data_dir = "/scratch/ti12/misc/for_Ben"

In [ ]:
word_glob = glob(f"{data_dir}/word_times/*")

In [ ]:
from scipy.io import loadmat, savemat

In [ ]:
for idx, patient in enumerate(patients):
    word_mat = loadmat(word_glob[idx])
    word_times[patient] = word_mat

In [ ]:
for idx, patient in enumerate(patients):
    spike_path = glob(f'{data_dir}/spike_trains/*{patient}*.mat')[0]

In [ ]:
for idx, patient in enumerate(patients):
    spike_path = glob(f'{data_dir}/spike_trains/*{patient}*.mat')[0]
    with h5py.File(spike_path, 'r') as f:
        chan_info[patient] = f['chan'][:].squeeze()
        sparse_mat = f["spikes"]
        data = sparse_mat["data"][:].flatten()  # Extract nonzero values
        ir = sparse_mat["ir"][:].flatten()      # Row indices
        jc = sparse_mat["jc"][:].flatten()      # Co

        # Determine the shape of the matrix (from jc array)
        num_rows = int(ir.max() + 1)  # Max row index + 1
        num_cols = len(jc) - 1   # The number of columns (jc's length - 1)

        # Create the scipy.sparse.csc_matrix
        sparse_matrix = sp.csc_matrix((data, ir, jc), shape=(num_rows, num_cols))
        spike_trains[patient] = sparse_matrix
    break

In [ ]:
f_96 = h5py.File(yex_paths[0])
f_99 = h5py.File(yex_paths[1])

In [ ]:
sparse_mat = f_96["spikes96"]
chan = f_96["chan96"][:]
data = sparse_mat["data"][:].flatten()  # Extract nonzero values
ir = sparse_mat["ir"][:].flatten()      # Row indices
jc = sparse_mat["jc"][:].flatten()      # Co

# Determine the shape of the matrix (from jc array)
num_rows = int(ir.max() + 1)  # Max row index + 1
num_cols = len(jc) - 1   # The number of columns (jc's length - 1)

# Create the scipy.sparse.csc_matrix
sparse_matrix_96 = sp.csc_matrix((data, ir, jc), shape=(num_rows, num_cols))

sparse_mat = f_99["spikes"]

data = sparse_mat["data"][:].flatten()  # Extract nonzero values
ir = sparse_mat["ir"][:].flatten()      # Row indices
jc = sparse_mat["jc"][:].flatten()      # Co

# Determine the shape of the matrix (from jc array)
num_rows = int(ir.max() + 1)  # Max row index + 1
num_cols = len(jc) - 1   # The number of columns (jc's length - 1)

# Create the scipy.sparse.csc_matrix
sparse_matrix_99 = sp.csc_matrix((data, ir, jc), shape=(num_rows, num_cols))

sparse_matrix = sp.hstack((sparse_matrix_96, sparse_matrix_99))

f_96.close()
f_99.close()

In [ ]:
spikes_trains["YEX"] = sparse_matrix
chan_info["YEX"] = chan.squeeze()

In [ ]:
# now get ofc indices
ofc_indices = {}
for patient in patients:
    try:
        micro_df = pd.read_csv(f"{data_dir}/micro_data/{patient}_micro_info.csv", index_col=0)
        ofc_indices[patient] = micro_df["region_symbol"] == "OFC"
    except Exception as e:
        print(e)
    break

In [ ]:
ofc_indices["UTA"] = np.array([0, 1, 2, 3, 4, 5, 6, 7])
ofc_indices["UTC"] = np.array([0, 1, 2, 3, 4, 5, 6, 7])
ofc_indices["UTE"] = np.array([0, 1, 2, 3, 4, 5, 6, 7])
ofc_indices["YEW"] = np.array([24, 25, 26, 27, 28, 29, 30, 31, 56, 57, 58, 59, 60, 61, 62, 63])

In [ ]:
chan_masks = {}
for patient in patients:
    chan_masks[patient] = np.isin(chan_info[patient] - 1, ofc_indices[patient])

In [ ]:
word_mats = {}
dur_mats = {}

In [ ]:
for patient in patients:
    pt_spike_list = []
    pt_dur_list = []
    pt_data = word_times[patient]
    if patient == "YFN":
        pt_data = np.delete(pt_data, np.arange(2262, 2275))
    word_stack = deque([re.sub(r"[^\w']+$", "", word.item()).lower() for word in pt_data["text"].squeeze() if word != "NaN"])
    pt_spike_train = spike_trains[patient][chan_masks[patient]]
    jdx = 0
    for idx in range(len(word_list)):
        if not word_stack:
            pt_spike_list.append(np.full(pt_spike_train.shape[0], np.nan))
            pt_dur_list.append(np.nan)
        elif word_stack[0] == word_list[idx]:
            start = np.round(pt_data["onset"].squeeze()[jdx].item() + 80).astype(int)
            end = np.round(pt_data["offset"].squeeze()[jdx].item()).astype(int)
            dur = end - start
            counts = pt_spike_train[:, start:end].toarray().sum(axis=1)
            pt_spike_list.append(counts)
            pt_dur_list.append(dur)
            word_stack.popleft()
            jdx += 1
        else:
            pt_spike_list.append(np.full(pt_spike_train.shape[0], np.nan))
            pt_dur_list.append(np.nan)
    print("length of word stack (should be empty!!):", len(word_stack))
    pt_spikes[patient] = np.vstack(pt_spike_list)
    pt_durs[patient] = np.array(pt_dur_list)
